# 使用Cython加密

```python
# -*- coding: utf-8 -*-
from Cython.Build import cythonize
from Cython.Distutils import build_ext
from setuptools import setup, Extension

# pattern is avilable: * or **.
extensions = [
    Extension('src.services.*', ['src/services/*.py']),
    Extension('src.views.*', ['src/views/*.py'])
]

setup(
    name="aps_ama_project",
    build_dir='build',
    # language_level represents the python version, and python3 writes 3.
    ext_modules=cythonize(
        extensions,
        compiler_directives={'language_level': 3, 'always_allow_keywords': True},
        # excluded specified files.
        exclude=['**/__init__.py']
    ),
    script_args=["build_ext", "--inplace"],
    cmdclass=dict(build_ext=build_ext)
)
```

文件批量重命名：`find . -name '*.so'|awk -F ".cpython-38-x86_64-linux-gnu." '{print $0,$1"."$2}'|xargs -n2 mv`


## Linux下创建Daemon进程方式

In [ ]:
"""利用tornado在windows上部署高并发应用"""

import os
import sys
import atexit
from multiprocessing import Process
from tornado.wsgi import WSGIContainer
from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop
from run import app


def _run_server():
    http_server = HTTPServer(WSGIContainer(app))
    http_server.listen(5000)
    IOLoop.instance().start()

def run(pid_file=None):
    Process(target=_run_server, daemon=True).start()
    pid = os.fork()
    if pid:
        sys.exit(0)
    # os.chdir()
    # os.umask(0)
    os.set_sid()
    _pid = os.fork()
    if _pid:
        sys.exit(0)
    
    sys.stderr.flush()
    sys.stdout.flush()

    # 写入pid文件
    if pid_file:
        with open(pid_file, 'w+') as f:
            f.write(str(os.getpid()))
        # 注册退出函数，进程异常退出时移除pid文件
        atexit.register(os.remove, pid_file)

if __name__ == "__main__":
    run()

## gevent启动Flask App

```python
from gevent import monkey
from gevent.pywsgi import WSGIServer
from geventwebsocket.handler import WebSocketHandler

monkey.patch_all()

app = Flask(__name__)

if __name == "__main__":
    server = WSGIServer(('', 5000), app, handler_class=WebSocketHandler)
    server.serve_forever()
```

## 后台daemon部署

```python
import subprocess
from src.settings import PID_FILE

DETACHED_PROCESS = 8

if __name__ == "__main__":
    p = subprocess.Popen("python deploy.py",
            creationflags=DETACHED_PROCESS,
            close_fds=True
        )
    with open(PID_FILE, 'w') as f:
        f.write(str(p.pid))
    print(f"Server Running: PID {p.pid} Bind http://0.0.0.0:5000")
```

# GIT

- git log --graph stash@{0}可以看出untracked files的提交位置；
- git show stash@{0}^3:file_dir |git apply: 可以将特定的文件apply；

# Sqlalchemy表反射

```
# reflect原表
meta_obj = MetaData().reflect(engine)
table = meta_obj.tables[table_name]
# 构造新表
temp_table = Table(table.name, MetaData())
for column in table.columns:
    if column.name in available_columns:
        temp_table.append_column(column.copy())
compiled_sql = temp_table.insert().compile(target_engine)
```

In [2]:
# mysql中变量： secure_file_priv 配置后可能会引起一些问题
